# Data Mining for Business Analytics
## Homework 4 Solutions

Student Name: Solutions

Student Netid: Solutions
***

### Part 1: Text
Answer each of the following. **`[`*`10 points each`*`]`**

1\. What are two problems with using text in machine learning? Why are these not issues when using numeric features as we have been using throughout class so far?

Text data is **unstructured** and **dirty** (a third option would be it **exists in context**, a fourth option would be **numeric weights on text data make no sense**). 

Documents can have varying lengths and be organized in many different ways. For example, we can have books with chapters, simple sentences, short Tweets, Wikipedia articles with sections. The features we have been using up until now were highly structured and were already in a "table" of data with records and features. Text data does not naturally look like this. Additionally, language itself is highly unstructured. English has many exceptions compared to rules!

Text is dirty. Words can be spelled incorrectly, they can be capitalized in different ways, acronyms and synonyms and confuse us.

2\. What is the name of the simplified approach we saw in class for dealing with text that ignores word order and sentence structure? What is one way of correcting for this simplification and how does it work? Why does it help us get some sense of word order? Why is it not a perfect solution?

Treating text as a "**bag of words**" will destroy any sense of word order and sentence structure. However, it makes it much easier to deal with text. We can simply count word frequency or occurence and use these as simple features. One way to correct for this simplification is to use **n-grams**. An n-gram will allow us to capture some of the word order that was in our text. For example, in the sentence "The movie is not bad.", instead of treating "not" and "bad" separately, we can use bigrams to account for "not bad" as a single token. This is not perfect since it can not capture all order without using a very high number n-gram!

3\. Some words will be more important that others when building text models. What are two ways of dealing with different levels of importance for words? *Hint: in class we saw one way to totally ignore certain words and one way to downplay particular words. Both of these dealt with "common" words.*

Two ways of dealing with importance of separate words are to use **stop word lists** and **tf-idf counts**. A stop word list will help us remove common words that have no real meaning in text classification problems. Words like "the", "a", and "is" can often be excluded. Tf-idf counts will give importance to words that appear frequently *within* a document but penalize words that appear frequently *across* documents.

***

### Part 2: Naïve Bayes
1\. From your reading you know that the naive Bayes classifier works by calculating the conditional probabilities of each feature, $e_i$, occuring with each class $c$ and treating them independently. This results in the probability of a certain class occuring given a set of features, or a piece of evidence, $E$, as

$$P(c \mid E) = \frac{p(e_1 \mid c) \cdot p(e_2 \mid c) \cdot \cdot \cdot p(e_k \mid c) \cdot p(c)}{p(E)}.$$

The conditional probability of each piece of evidence occuring with a given class is given by

$$P(e_i \mid c) = \frac{\text{count}(e_i, c)}{\text{count}(c)}.$$

In the above equation $\text{count}(e_i, c)$ is the number of documents in a given class that contain feature $e_i$ and $\text{count}(c)$ is the number of documents that belong to class $c$. 

A common variation of the above is to use Laplace (sometimes called +1) smoothing. Recall the use of Laplace smoothing introduced toward the end of Chapter 3 in the section Probability Estimation. This is done in sklearn by setting `alpha=1` in the `BernoulliNB()` function (this is also the default behavior). The result of Laplace smoothing will slightly change the conditional probabilities,

$$P(e_i \mid c) = \frac{\text{count}(e_i, c) + 1}{\text{count}(c) + 2}.$$

In no more than **one paragraph**, describe why this is useful. Try to think of a case when not using Laplace smoothing would result in "bad" models. Try to give an example. *We discussed this in class. Do you think probabilities or counts of zero might be a problem?* Be precise. **`[`*`15 points`*`]`**

If we encounter a brand new word we have never seen before, then the probability will be zero! Since we are multiplying many probabilities together, having a single zero will zero out the entire measure. When expanding to real world problems where new records need to be classsified, we will often encounter brand new words we know nothing about --- English is a very large language! Using Laplace smoothing will result in us never having a zero probability. This is done by simply adding a small amount of "noise" to the numerator and denominator.

***

### Part 3: Feature engineering
Answer each of the following. **`[`*`10 points each`*`]`**

1\. Why are non numeric features a problem during the modeling phase? Are non numeric features a problem for all types of models?

Categorical features can be used fairly easily in a tree structured classifier. The model will simply try splitting on each category level instead of trying to create many splits on numeric data. However, in linear discriminant models, the use of categorical features without any modification is not so straight forward. Consider a linear model,

$$f(x) = b + w_1x_1 + w_2x_2 + ...$$.

If the feature $x_1$ is a categorical feature like "marital status", then applying a numeric weight $w_1$ will make no sense. You can't multiple a number by a word like "married".

2\. Categorical variables that can be thought of as scales (e.g., a measure of satisfaction) can be mapped to a numeric scale. What is an issue that we face when performing this mapping? What is an alternative to creating a mapping?

If we want to use a numeric mapping for a categorical scale feature, such as satisfaction level, we need to come up with a numeric map that works. This is not always easy. For example, if we have 5 levels, "Very Low", "Low", "Neutral", "High", "Very High", how can we justify using the off-by-one mapping: -2, -1, 0, 1, 2 instead of one with a wider spread: -10, -2, 0, 2, 10, or a non symmetric map: -10, -5, 0, 1, 5? In reality, mapping may be impossible to verify! One alternative is to simply binarize the feature and turn each level into it's own feature (leaving one out, of course).

***

### Part 4: Text mining
We are going to use some data from IMDB movie review to predict if reviews are positive or negative. The file `data/imdb.csv` has two columns:

- `Text`: The text from the IMDB review
- `Class`: Set to 1 if the review is positive and 0 if it is negative

In [1]:
# Run this cell to import the stuff you will need
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation

1\. Read in the data located in `data/imdb.csv`. Don't forget to set the `quotechar` to `"` and `escapechar` to `\`. **`[`*`5 points`*`]`**

In [2]:
data = pd.read_csv("data/imdb.csv", quotechar="\"", escapechar="\\")

2\. Put your text data into a variable called `X_text` and your target variable into a variable called `Y`.

In [3]:
X_text = data['Text']
Y = data['Class']

3\. What is the base rate of this data set? **`[`*`5 points`*`]`**

In [4]:
print "The baserate is " + str(Y.mean())

The baserate is 0.498882221438


4\. What is the 5-fold area under the ROC curve for a logistic regression model with default regularization? Your features should consist of all **tf-idf count** text features **exluding** English stop words and keeping the original capitalization (i.e., turn lowercase off). **`[`*`10 points`*`]`**

In [5]:
# Create a vectorizer that will track text as binary features
count_vectorizer = TfidfVectorizer(stop_words="english", lowercase=False)

# Let the vectorizer learn what tokens exist in the text data
count_vectorizer.fit(X_text)

# Turn these tokens into a numeric matrix
X = count_vectorizer.transform(X_text)

# Create a model
logistic_regression = LogisticRegression()

# Use this model and our data to get 5-fold cross validation AUCs
aucs = cross_validation.cross_val_score(logistic_regression, X, Y, scoring="roc_auc", cv=5)

# Print out the average AUC rounded to three decimal points
print "Area under the ROC curve for our classifier is " + str(np.mean(aucs))

Area under the ROC curve for our classifier is 0.937885927427


5\. What is the 5-fold area under the ROC curve for a naïve Bayes model with default smoothing? Your features should consist of all **binary count** text features **including** English stop words, converting everything to lowercase, and contain unigrams and bigrams. **`[`*`10 points`*`]`**

In [7]:
# Create a vectorizer that will track text as binary features
count_vectorizer = CountVectorizer(binary=True, ngram_range=(1,2))

# Let the vectorizer learn what tokens exist in the text data
count_vectorizer.fit(X_text)

# Turn these tokens into a numeric matrix
X = count_vectorizer.transform(X_text)

# Create a model
naive_bayes = BernoulliNB()

# Use this model and our data to get 5-fold cross validation AUCs
aucs = cross_validation.cross_val_score(naive_bayes, X, Y, scoring="roc_auc", cv=5)

# Print out the average AUC rounded to three decimal points
print "Area under the ROC curve for our classifier is " + str(np.mean(aucs))

Area under the ROC curve for our classifier is 0.925866707355


6\. Of these two models, which would you prefer? Why? Are there other things we should have tried? What else might you want to do before choosing your final model? **`[`*`5 points`*`]`**

From what I have of these two models, I would go with the logistic regression in question 4. It outperforms the naive Bayes by a fair amount. To expand on this, however, I would definitely want to try two main things: (1) different combinations of n_grams, stop words, lowercase transformations for both model types --- including n-grams in the logistic regression will probably make it perform even better, and (2) different levels of complexity for each model type. 